In [1]:
import os, sys, re
import numpy as np
import pandas as pd
from pathlib import Path
from functools import reduce
from pandas import HDFStore,DataFrame
# define the current path (notebooks in lab_utils)
currpath = os.getcwd()
labutilspath = str(Path(currpath).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

# define paths
basepath = '/media/damiansa/data/lab/data/characterization/'

# set the asdatapath accordingly (where is the atuoscan data inside basedatapath?)
asdatapath   = 'autoscan'

#sample selection and instance
tag = 'sh_001'
instance = 'before'

# set datapath
datapath = os.path.join(basepath, asdatapath)

pp = autoscan.postprocess(labutilspath=labutilspath)
pp.debug = False

In [2]:
# load the summary dataframe with all measurement
summary = pd.read_csv(os.path.join(datapath,'summary.csv'))
# remove ztop and zbottom
t = summary['relroot'].apply(lambda x: len(re.findall(r'ztop|zbottom', x))==0)
summary = summary.loc[t,:].copy()

sample_info = pp._subset_info(summary, instance = instance, sample_tag = tag)

In [3]:

# dfs = []
# for s in sample_info.itertuples(index=False):
#     relpath = getattr(s, 'relroot')
#     probe   = getattr(s, 'probe')
#     data = pp.read_data(os.path.join(datapath, relpath), probe = probe)
#     h = pp.probe_settings[probe]['h']
#     data = data.iloc[:,:h]
#     if probe == 'vel':
#         for x in data.angle.unique():
#             dfs.append(data.loc[data['angle'] == x,:].copy().drop(columns = 'angle'))
#     else:
#         dfs.append(data)

# df_merged = reduce(lambda left,right: pd.merge(left,right,on=['x','y'], how ='inner'), dfs)
# df_merged = pp._enforce_float(df_merged)
# df_merged['tag'] = tag
# df_merged['instance'] = instance


In [4]:
save = False
fullset = []
problem = {}
problem_merged = []
ns = 0
for instace in ['before', 'after']:
    for tag in summary['sample_tag'].unique():
        s1 = pp._subset_info(summary, instance = instance, sample_tag = tag)
        if len(s1)>0:
            family = s1['sample_family'].unique()[0]
            code   = s1['sample_code'].unique()[0]
            for subtag in s1['subsample_tag'].unique():
                s2 = pp._subset_info(s1, subsample_tag = subtag, sample_code = code, sample_family = family)
                for kside, side in enumerate(s2['side'].unique()):
                    s3 = pp._subset_info(s2, side = side)
                    probes  = s3['probe'].unique()
                    nprobes = len(probes)
                    if nprobes>1 and ('ftir' in probes):
                        dfs = []
                        for meas in s3.itertuples(index=False):
                            relpath  = getattr(meas, 'relroot')
                            filepath = os.path.join(datapath, relpath)
                            probe    = getattr(meas, 'probe')
                            
                            data     = pp.read_data(filepath, probe = probe)
                            
                            h    = pp.probe_settings[probe]['h']
                            data = data.iloc[:,:h]

                            if probe == 'vel':
                                for k,x in enumerate(data.angle.unique()):
                                    colnames = ['vp_'+str(k), 'vs_'+str(k)]
                                    data_temp = data.loc[data['angle'] == x,:].copy().drop(columns = 'angle')
                                    data_temp.columns = ['x','y'] + colnames
                                    dfs.append(data_temp)
                                    del data_temp
                            else:
                                dfs.append(data)

                        df_merged = reduce(lambda left,right: pd.merge(left,right,on=['x','y'], how ='inner'), dfs)
                        df_merged = pp._enforce_float(df_merged)

                        if df_merged.shape[0]>0 and len(df_merged.iloc[:,1754:].columns.values)>1:
                            ns += df_merged.shape[0]
                            df_merged['family']   = family
                            df_merged['code']     = code
                            df_merged['tag']      = tag
                            df_merged['subtag']   = subtag
                            df_merged['instance'] = instance
                            df_merged['side']     = kside
                            
                            if save:
                                outpath = pp._set_outpath(datapath,tag)
                                outname = pp._set_outfilename(tag, subtag, side, instance, 'fullset')
                                pp.save_data(df_merged, savepath = outpath,  savename = outname)
                        
                            fullset.append(df_merged.iloc[:,2:])



In [5]:
ns
compiledinfocols = ['family', 'code', 'tag', 'subtag', 'side', 'instance']
compileddatacols = pp.probe_settings['ftir']['names'][2:] + ['perm', 'vp_0', 'vs_0', 'vp_1', 'vs_1', 'e_star']  
compiledfullcols = compiledinfocols + compileddatacols
# fullset[1].columns[1750:]
dfc = pd.concat(fullset, sort = False, axis = 0, ignore_index=True, join = 'outer')
dfc = dfc.loc[:,compiledfullcols].copy()
dfc.loc[:,compileddatacols] = dfc.loc[:,compileddatacols].apply(pd.to_numeric,errors='coerce').copy()

In [6]:
savefile = os.path.join(datapath, 'as_dataset.h5')
dfc.to_hdf(savefile, key = 'data', mode = 'w')

In [13]:
import deepdish as dp
dp.io.save(os.path.join(datapath, 'data.h5'), dfc.loc[:, compileddatacols])


In [14]:
dp.io.save(os.path.join(datapath, 'descriptions.h5'), dfc.loc[:, compiledinfocols])

In [15]:
dp.io.save(os.path.join(datapath, 'compiled.h5'), dfc)

/home/damiansa/sandbox/opt/anaconda/lib/python3.7/site-packages/deepdish/io/hdf5io.py:246: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif _pandas and isinstance(level, (pd.DataFrame, pd.Series, pd.Panel)):
